### TỰ ĐỘNG CÀO DATA TỪ TRANG WEB 24HMONEY

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Khởi tạo trình duyệt Chrome với webdriver_manager
def initialize_driver():
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service)

# Hàm để chuyển đổi dữ liệu thành số nếu cần
def convert_to_number(value):
    if ',' in value:
        temp = value.replace(',', '')
        try:
            return int(temp)
        except ValueError:
            try:
                return float(temp)
            except ValueError:
                return value
    if ',' not in value:
        try:
            return int(value)
        except ValueError:
            try:
                return float(value)
            except ValueError:
                return value


# Hàm để cào dữ liệu từ một loại filter
def scrape_data(driver, filter_text):
     
    # Nhảy tới mục lớn cần cào 
    try:
        filter_element = driver.find_element(By.XPATH, f"//div[contains(text(), '{filter_text}')]")
        filter_element.click()
        time.sleep(2)  # Chờ trang tải
        print(f"Đã chuyển sang filter '{filter_text}'")
    except Exception as e:
        print(f"Không thể chuyển sang filter '{filter_text}': {e}")
        driver.quit()  # Thoát khỏi trình duyệt nếu không thể chuyển sang filter
        
        
    list_ = ['Theo Quý', 'Theo Năm']
    for index, text in enumerate(list_):
        # Chọn mục nhỏ cần cào     
        try:
            filter_element = driver.find_element(By.XPATH, f"//div[contains(text(), '{text}')]")
            filter_element.click()
            time.sleep(2)  # Chờ trang tải
            print(f"Đã chuyển sang filter '{text}'")
        except Exception as e:
            print(f"Không thể chuyển sang filter '{text}': {e}")
            driver.quit()  # Thoát khỏi trình duyệt nếu không thể chuyển sang filter

        if index == 0:
            all_data_df = None

        while True:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            table = soup.find('table')

            if table:
                current_page_df = pd.DataFrame()

                rows = table.find_all('tr')

                for i, row in enumerate(rows):
                    if i == 0:
                        header_cols = row.find_all('th')
                        headers = [col.text.strip() for col in header_cols]
                    else:
                        processing_row = []
                        cols = row.find_all('td')
                        for col_index, col in enumerate(cols):
                            text = col.text.strip()
                            if col_index != 0:
                                processing_row.append(convert_to_number(text))
                            else:
                                processing_row.append(text)
                        current_page_df = pd.concat([current_page_df, pd.DataFrame([processing_row], columns=headers)], ignore_index=True)

                if all_data_df is not None:
                    current_page_df = current_page_df.iloc[:, 1:]

                if all_data_df is None:
                        all_data_df = current_page_df
                else:
                    all_data_df = pd.concat([all_data_df, current_page_df], axis=1)

            try:
                button_next_page = driver.find_element(By.XPATH, "//div[@class='pager-button' and contains(text(), 'Trang sau')]")
                button_next_page.click()
                time.sleep(2)  # Chờ trang tải
                print("Chuyển sang trang tiếp theo")
            except Exception as e:
                print(f"Không tìm thấy nút 'Trang sau': {e}")
                break

    return all_data_df



# Hàm đổi tên các cột cho đồng bộ
def replace_columns_name(column_name):
    return column_name.replace('Q', 'Quý ').replace('/', ' - 20')



# Hàm sắp xếp tên các cột theo thứ tự Qúy -> Năm
def sort_columns(column_list):
    sorted_columns = []
    sorted_columns.append(column_list[0])
    column_list = column_list[1:]
    
    quarter_col = []
    year_col = []
    # Phân loại các cột
    for col in column_list:
        if col.startswith('Quý'):
            quarter_col.append(col)
        else:
            year_col.append(col)
    
    # Sắp xếp các cột "Quý" và "Năm" theo thứ tự
    year_col.sort(key=lambda x: int(x))
    quarter_col.sort(key=lambda y: (int(y.split(' ')[1]), int(y.split(' ')[-1])))  # key trong hàm sort là truyền vào 1 hàm khác

    # Sắp xếp cho quý và năm xen kẽ nhau
    for year in year_col:
        # Thêm các các quý vào danh sách kết quả
        sorted_columns.extend(q for q in quarter_col if q.endswith(year))
        # Thêm các năm vào ds
        sorted_columns.append(year)
    
    return sorted_columns
    


############################################################################################################################


# Khởi tạo trình duyệt và URL
driver = initialize_driver()
url = 'https://24hmoney.vn/stock/PGB/financial-report?view=2&period=4'
driver.get(url)


filter_ = ['Kết quả KD', 'Cân đối KT', 'LC Tiền tệ']
file_name = ['IS', 'BS', 'CF']
name_company = url.split('/')[-2]
sheet_name_ =[]


for index, session in enumerate(filter_):
    # Lấy dữ liệu từ các filter
    all_data_df = scrape_data(driver, session)


    # Xóa các cột có dấu % 
    not_col_percent = [col for col in all_data_df
                        if '%' not in col]
    all_data_df = all_data_df[not_col_percent]


    # Loại bỏ các data có giá trị là N/A
    all_data_df.replace('N/A', '', inplace=True)


    # Đổi tên các cột sao cho đồng nhất
    new_columns ={}
    for col in all_data_df.columns:
        new_columns[col] = replace_columns_name(col)
    all_data_df.rename(columns=new_columns, inplace=True)


    # Sắp xếp các quý và năm theo format
    column_list = [col for col in all_data_df.columns]
    sorted_columns = sort_columns(column_list)
    all_data_df_full = all_data_df[sorted_columns]

    
    
    if index ==0:
        all_data_df_full.to_excel(f'{name_company}_combine.xlsx', sheet_name=f'{file_name[index]}', index=False)
        sheet_name_.append(file_name[index])   
    else:
        file_path = f'D:\Công việc\My Work\{name_company}_combine.xlsx'
              
        # Kết hợp dữ liệu từ file mới với dữ liệu từ file gốc và ghi đè lên file gốc
        if file_name[index] not in sheet_name_:  # Kiểm tra xem sheet có trong danh sách không cần sửa không
            with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
                all_data_df_full.to_excel(writer, sheet_name=f'{file_name[index]}', index=False)
            sheet_name_.append(file_name[index])



# Đóng trình duyệt khi đã hoàn thành
driver.quit()

<string>:193: SyntaxWarning: invalid escape sequence '\{'
<>:193: SyntaxWarning: invalid escape sequence '\{'
<>:193: SyntaxWarning: invalid escape sequence '\C'
<string>:193: SyntaxWarning: invalid escape sequence '\{'
<>:193: SyntaxWarning: invalid escape sequence '\{'
<>:193: SyntaxWarning: invalid escape sequence '\C'
C:\Users\nhuva\AppData\Local\Temp\ipykernel_33004\3300998037.py:193: SyntaxWarning: invalid escape sequence '\{'
  file_path = f'D:\Công việc\My Work\{name_company}_combine.xlsx'
C:\Users\nhuva\AppData\Local\Temp\ipykernel_33004\3300998037.py:193: SyntaxWarning: invalid escape sequence '\C'
  file_path = f'D:\Công việc\My Work\{name_company}_combine.xlsx'
C:\Users\nhuva\AppData\Local\Temp\ipykernel_33004\3300998037.py:193: SyntaxWarning: invalid escape sequence '\{'
  file_path = f'D:\Công việc\My Work\{name_company}_combine.xlsx'
C:\Users\nhuva\AppData\Local\Temp\ipykernel_33004\3300998037.py:193: SyntaxWarning: invalid escape sequence '\C'
  file_path = f'D:\Công vi

OSError: [WinError 193] %1 is not a valid Win32 application